In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
from tensorflow.keras import datasets, layers, models

In [2]:
train_data_dir = '../data/localdata/chest_xray/train'
test_data_dir = '../data/localdata/chest_xray/test'

generator =ImageDataGenerator(rescale=1/255, validation_split=.25)

train_generator= generator.flow_from_directory(
        train_data_dir, target_size=(64, 64), batch_size=32, color_mode='grayscale', subset='training', class_mode='binary')
validation_set=generator.flow_from_directory(
        train_data_dir, target_size=(64, 64), batch_size=32, color_mode='grayscale', subset='validation', class_mode='binary')
test_generator = generator.flow_from_directory(
        test_data_dir, target_size=(64, 64), batch_size=32, color_mode='grayscale', class_mode='binary')

train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

Found 3913 images belonging to 2 classes.
Found 1303 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
train_images.shape

(32, 64, 64, 1)

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [5]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision
#model.compile(optimizer=tf.keras.optimizers.Adam(),
             # loss=tf.keras.losses.BinaryCrossentropy(),
             # metrics=tf.keras.metrics.Precision())

In [9]:
#model.fit(train_generator, epochs=10, batch_size=200, validation_data=validation_set)

Epoch 1/10
123/123 [==============================] - 124s 1s/step - loss: 0.2982 - precision: 0.8780 - val_loss: 0.1712 - val_precision: 0.9690
Epoch 2/10
123/123 [==============================] - 92s 747ms/step - loss: 0.1223 - precision: 0.9660 - val_loss: 0.1277 - val_precision: 0.9679
Epoch 3/10
123/123 [==============================] - 91s 737ms/step - loss: 0.0941 - precision: 0.9756 - val_loss: 0.1031 - val_precision: 0.9741
Epoch 4/10
123/123 [==============================] - 97s 788ms/step - loss: 0.0828 - precision: 0.9793 - val_loss: 0.0959 - val_precision: 0.9752
Epoch 5/10
123/123 [==============================] - 107s 871ms/step - loss: 0.0752 - precision: 0.9804 - val_loss: 0.1085 - val_precision: 0.9694
Epoch 6/10
123/123 [==============================] - 118s 962ms/step - loss: 0.0701 - precision: 0.9831 - val_loss: 0.1102 - val_precision: 0.9850
Epoch 7/10
123/123 [==============================] - 66s 536ms/step - loss: 0.0576 - precision: 0.9879 - val_loss: 0.

In [47]:
print('''Epoch 10/10
123/123 [==============================] - 59s 484ms/step - loss: 0.0388 - precision: 0.9910 - val_loss: 0.1154 - val_precision: 0.9831''')

Epoch 10/10
123/123 [==============================] - 59s 484ms/step - loss: 0.0388 - precision: 0.9910 - val_loss: 0.1154 - val_precision: 0.9831


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

simple_model = models.Sequential()
simple_model.add(layers.Flatten())
simple_model.add(Dense(12, activation='relu'))
simple_model.add(Dense(1, activation='sigmoid'))

In [49]:
simple_model.compile(optimizer='Adam',
              loss='BinaryCrossentropy',
              metrics='Precision')
                 
#simple_model.fit(train_generator, epochs=10, validation_data=validation_set)
print('''Epoch 10/10
123/123 [==============================] - 79s 643ms/step - loss: 0.1684 - precision_6: 0.9665 - val_loss: 0.1915 - val_precision_6: 0.9673''')

Epoch 10/10
123/123 [==============================] - 79s 643ms/step - loss: 0.1684 - precision_6: 0.9665 - val_loss: 0.1915 - val_precision_6: 0.9673


In [51]:
simple_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, None)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 12)                49164     
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 13        
Total params: 49,177
Trainable params: 49,177
Non-trainable params: 0
_________________________________________________________________


In [52]:
model2 = models.Sequential()
model2.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 1)))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(optimizer='Adam',
              loss='BinaryCrossentropy',
              metrics='Precision')

In [53]:
model2.fit(train_generator, epochs=10, validation_data=validation_set)

Epoch 1/10
123/123 [==============================] - 95s 773ms/step - loss: 0.3095 - precision: 0.8957 - val_loss: 0.1604 - val_precision: 0.9316
Epoch 2/10
123/123 [==============================] - 99s 804ms/step - loss: 0.1101 - precision: 0.9713 - val_loss: 0.1779 - val_precision: 0.9057
Epoch 3/10
123/123 [==============================] - 107s 869ms/step - loss: 0.0797 - precision: 0.9788 - val_loss: 0.1271 - val_precision: 0.9381
Epoch 4/10
123/123 [==============================] - 140s 1s/step - loss: 0.0683 - precision: 0.9835 - val_loss: 0.1035 - val_precision: 0.9615
Epoch 5/10
123/123 [==============================] - 95s 773ms/step - loss: 0.0718 - precision: 0.9791 - val_loss: 0.0931 - val_precision: 0.9843
Epoch 6/10
123/123 [==============================] - 95s 776ms/step - loss: 0.0547 - precision: 0.9852 - val_loss: 0.0898 - val_precision: 0.9744
Epoch 7/10
123/123 [==============================] - 87s 710ms/step - loss: 0.0391 - precision: 0.9904 - val_loss: 0.1

In [82]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint
model3 = models.Sequential()
model3.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 1)))
model3.add(layers.MaxPooling2D(pool_size=(2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))

model3.compile(optimizer='Adam',
              loss='BinaryCrossentropy',
              metrics=['Precision', 'BinaryAccuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           verbose=1, patience=3,
                           mode='min')
checkpoint = ModelCheckpoint('../data/localdata/chest_xray/BestCNNModel', monitor='val_loss',
    save_best_only= True)

results = model3.fit(train_generator, epochs=10, validation_data=validation_set,
                    callbacks=[early_stop, checkpoint])

Epoch 1/10
123/123 [==============================] - 70s 565ms/step - loss: 0.2670 - precision: 0.9004 - binary_accuracy: 0.8916 - val_loss: 0.1395 - val_precision: 0.9620 - val_binary_accuracy: 0.9478
Epoch 2/10
123/123 [==============================] - 101s 823ms/step - loss: 0.1081 - precision: 0.9702 - binary_accuracy: 0.9599 - val_loss: 0.1108 - val_precision: 0.9569 - val_binary_accuracy: 0.9563
Epoch 3/10
123/123 [==============================] - 75s 606ms/step - loss: 0.0887 - precision: 0.9766 - binary_accuracy: 0.9663 - val_loss: 0.1197 - val_precision: 0.9799 - val_binary_accuracy: 0.9524
Epoch 4/10
123/123 [==============================] - 93s 758ms/step - loss: 0.0744 - precision: 0.9811 - binary_accuracy: 0.9732 - val_loss: 0.1545 - val_precision: 0.9249 - val_binary_accuracy: 0.9348
Epoch 5/10
123/123 [==============================] - 81s 662ms/step - loss: 0.0713 - precision: 0.9808 - binary_accuracy: 0.9742 - val_loss: 0.1855 - val_precision: 0.9954 - val_binary_a